In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
from src.models.EMOS_global.EMOS_global_load_score import *
from src.models.EMOS_local.EMOS_local_load_score import *
from src.models.EMOS_local.EMOS_local_load_model import *
from src.models.EMOS_global.EMOS_global_load_model import *
from src.models.EMOS_local.EMOS_local_hyper_tune import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *
from src.models.EMOS_local.EMOS_local_train import *

2023-06-02 16:34:04.107136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-02 16:34:04.107157: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 1. Load Global Hyperparameter Scores

In [ ]:
EMOS_global_t2m_hyper_scores = EMOS_global_load_hyper_score("t2m")
EMOS_global_ws10_hyper_scores = EMOS_global_load_hyper_score("ws10")

In [ ]:
EMOS_global_t2m_hyper_scores

In [ ]:
EMOS_global_ws10_hyper_scores

### 2. Load Local Hyperparameter Scores

In [15]:
EMOS_local_t2m_hyper_scores = EMOS_local_load_hyper_score("t2m")
EMOS_local_ws10_hyper_scores = EMOS_local_load_hyper_score("ws10")

<IPython.core.display.Javascript object>

In [16]:
EMOS_local_t2m_hyper_scores

[[(32, 30, 0.01, 'Adam'), 0.3806851086899256],
 [(32, 30, 0.01, 'Adam'), 0.38007692008828503]]

<IPython.core.display.Javascript object>

In [17]:
EMOS_local_ws10_hyper_scores

[[(64, 30, 0.1, 'Adam'), 0.5994870296846597],
 [(32, 30, 0.1, 'Adam'), 0.5986838517471765]]

<IPython.core.display.Javascript object>

### 3. Scores for comparison:

In [18]:
# Load EMOS local scores
EMOS_local_t2m_scores = EMOS_local_load_score("t2m")
# EMOS_local_ws10_scores = EMOS_local_load_score("ws10")

# Load EMOS global scores
EMOS_global_t2m_scores = EMOS_global_load_score("t2m")
EMOS_global_ws10_scores = EMOS_global_load_score("ws10")

# Load CRPS scores
CRPS_global_t2m_scores = crps_load_lead_lat_lon("t2m")
CRPS_global_ws10_scores = crps_load_lead_lat_lon("ws10")

<IPython.core.display.Javascript object>

In [19]:
print("Temperature:")
for lead_time in range(len(EMOS_local_t2m_scores)):
    print(
        f"Lead {lead_time}:EMOS_loc: {EMOS_local_t2m_scores[lead_time].mean()}, EMOS_glob: {EMOS_global_t2m_scores[lead_time].mean()}, CRPS: {CRPS_global_t2m_scores[lead_time].mean()}"
    )

Temperature:
Lead 0:EMOS_loc: 0.4148695374260971, EMOS_glob: 0.4673401873320892, CRPS: 0.5084816639136492
Lead 1:EMOS_loc: 0.4570583921154468, EMOS_glob: 0.5297212701322601, CRPS: 0.5347707002046389
Lead 2:EMOS_loc: 0.4764913273333677, EMOS_glob: 0.5105776602793739, CRPS: 0.5150757989800734
Lead 3:EMOS_loc: 0.4853422424293378, EMOS_glob: 0.5187042066593011, CRPS: 0.5301602687833538
Lead 4:EMOS_loc: 0.5088349645341457, EMOS_glob: 0.5379253302273839, CRPS: 0.5624034501418446
Lead 5:EMOS_loc: 0.5484872642343575, EMOS_glob: 0.5924092410007133, CRPS: 0.6306282599852645
Lead 6:EMOS_loc: 0.5712123582837434, EMOS_glob: 0.5920924175090567, CRPS: 0.6316119778923832
Lead 7:EMOS_loc: 0.5754663166502546, EMOS_glob: 0.6020434806248538, CRPS: 0.6454345388357919
Lead 8:EMOS_loc: 0.6026712332169245, EMOS_glob: 0.6251029754078069, CRPS: 0.6802850949781106
Lead 9:EMOS_loc: 0.6562236252654708, EMOS_glob: 0.6920910937109535, CRPS: 0.7561511514525252
Lead 10:EMOS_loc: 0.6800776498304495, EMOS_glob: 0.698421

<IPython.core.display.Javascript object>

In [13]:
print("wind speed:")
for lead_time in range(len(EMOS_global_ws10_scores)):
    print(
        f"Lead {lead_time}: EMOS_glob: {EMOS_global_ws10_scores[lead_time].mean()}, CRPS: {CRPS_global_ws10_scores[lead_time].mean()}"
    )

wind speed:
Lead 0: EMOS_glob: 0.4687979519367218, CRPS: 0.4708288311958313
Lead 1: EMOS_glob: 0.5481671690940857, CRPS: 0.5626273155212402
Lead 2: EMOS_glob: 0.5383585095405579, CRPS: 0.5660684704780579
Lead 3: EMOS_glob: 0.5816507935523987, CRPS: 0.6244665384292603
Lead 4: EMOS_glob: 0.5986496806144714, CRPS: 0.6273626685142517
Lead 5: EMOS_glob: 0.6971593499183655, CRPS: 0.7419385313987732
Lead 6: EMOS_glob: 0.6992777585983276, CRPS: 0.7447009086608887
Lead 7: EMOS_glob: 0.7526292204856873, CRPS: 0.8054525852203369
Lead 8: EMOS_glob: 0.791255533695221, CRPS: 0.834025502204895
Lead 9: EMOS_glob: 0.8987031579017639, CRPS: 0.9581987261772156
Lead 10: EMOS_glob: 0.8913975954055786, CRPS: 0.9486294984817505
Lead 11: EMOS_glob: 0.9364757537841797, CRPS: 0.997710645198822
Lead 12: EMOS_glob: 0.9739682078361511, CRPS: 1.0248152017593384
Lead 13: EMOS_glob: 1.0816439390182495, CRPS: 1.147271752357483
Lead 14: EMOS_glob: 1.061387538909912, CRPS: 1.120965838432312
Lead 15: EMOS_glob: 1.0941627

<IPython.core.display.Javascript object>